### Please be aware that the official implementation of StyTr2 may not be able to used directly, as it may encounter errors related to gradient backpropagation failures. Please comment out the part after Ics in the stytr.py.


In [ ]:
import argparse
from pathlib import Path
import os
import torch
import torch.nn as nn
from PIL import Image
from os.path import basename
from os.path import splitext
from torchvision import transforms
from torchvision.utils import save_image
from function import calc_mean_std, normal, coral
import models.transformer as transformer
import models.StyTR as StyTR
import matplotlib.pyplot as plt
from matplotlib import cm
from function import normal
import numpy as np
import time
from collections import OrderedDict
new_state_dict = OrderedDict()

import clip
clip_model, preprocess = clip.load("ViT-B/32", device="cuda")

import cv2 as cv


In [ ]:
def test_transform(size, crop):
    transform_list = []
   
    if size != 0: 
        transform_list.append(transforms.Resize(size))
    if crop:
        transform_list.append(transforms.CenterCrop(size))
    transform_list.append(transforms.ToTensor())
    transform = transforms.Compose(transform_list)
    return transform
def style_transform(h,w):
    k = (h,w)
    size = int(np.max(k))
    print(type(size))
    transform_list = []    
    transform_list.append(transforms.CenterCrop((h,w)))
    transform_list.append(transforms.ToTensor())
    transform = transforms.Compose(transform_list)
    return transform

def content_transform():
    
    transform_list = []   
    transform_list.append(transforms.ToTensor())
    transform = transforms.Compose(transform_list)
    return transform

In [ ]:
content_size=512
style_size=512
crop='store_true'
save_ext='.jpg'
preserve_color='store_true'
alpha=10.0

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
vgg = StyTR.vgg
vgg.load_state_dict(torch.load('./experiments/vgg_normalised.pth'))
vgg = nn.Sequential(*list(vgg.children())[:44])

decoder = StyTR.decoder
Trans = transformer.Transformer()
embedding = StyTR.PatchEmbed()
decoder.eval()
Trans.eval()
vgg.eval()
print('init')

In [ ]:
state_dict = torch.load('experiments/decoder_iter_160000.pth')

for k, v in state_dict.items():
    #namekey = k[7:] # remove `module.`
    namekey = k
    new_state_dict[namekey] = v
decoder.load_state_dict(new_state_dict)

In [ ]:
new_state_dict = OrderedDict()
state_dict = torch.load('experiments/transformer_iter_160000.pth')
for k, v in state_dict.items():
    #namekey = k[7:] # remove `module.`
    namekey = k
    new_state_dict[namekey] = v
Trans.load_state_dict(new_state_dict)

In [ ]:
new_state_dict = OrderedDict()
state_dict = torch.load('experiments/embedding_iter_160000.pth')
for k, v in state_dict.items():
    #namekey = k[7:] # remove `module.`
    namekey = k
    new_state_dict[namekey] = v
embedding.load_state_dict(new_state_dict)

In [ ]:
# with torch.no_grad():
network = StyTR.StyTrans(vgg,decoder,embedding,Trans,None)

network.eval()

network.to(device)

print('init done')

In [ ]:
content_tf = test_transform(content_size, crop)
style_tf = test_transform(style_size, crop)

In [ ]:

content_path = 'content/content.jpg'
style_path = 'style/style.jpg'
output_path='/output/'

img_size = 

content = content_tf(Image.open(content_path).convert("RGB"))
h,w,c=np.shape(content)    
style = style_tf(Image.open(style_path).convert("RGB"))


style = style.to(device).unsqueeze(0)
style += torch.rand_like(style) / 1e7
content = content.to(device).unsqueeze(0)
content += torch.rand_like(content) / 1e7


style = nn.functional.interpolate(style, (img_size,img_size))
content = nn.functional.interpolate(content, (img_size,img_size))


style.requires_grad_(True)
style.retain_grad()
content.requires_grad_(True)
content.retain_grad()


word = 'puzzle'
word_original = 'oil painting'


word_token = clip.tokenize([word, word_original]).cuda()
text_feature = clip_model.encode_text(word_token)
text_feature_delta = text_feature[0:1] - text_feature[1:2]


eps = 

batch={}
batch['style'] = style
batch['content'] = content


batch['noise_style'] = torch.rand_like(style) / 1e7
batch['noise_content'] = torch.rand_like(content) / 1e7
output_gt = network(content,style).detach()


output_name_gt = '{:s}/{:s}_stylized_{:s}_{:s}_gt{:s}'.format(
        output_path, splitext(basename(content_path))[0],
        splitext(basename(style_path))[0], word if word is not None else '', save_ext
    )
save_image(output_gt, output_name_gt)
    


img_feature_old = clip_model.encode_image(torch.nn.functional.interpolate(\
                                               output_gt,(224,224)))

content_gt= batch['content'].clone().detach()
style_gt= batch['style'].clone().detach()

step=10
for i in range(step):
    batch['content'] = (batch['content'] + batch['noise_content']).detach() # noised input
    batch['content'].requires_grad_(True)
    batch['content'].retain_grad()

    batch['style'] = (batch['style'] + batch['noise_style']).detach()
    batch['style'].requires_grad_(True)
    batch['style'].retain_grad()
    
    output_previous = network(batch['content'] ,batch['style'])
    
    ########################### ATTACK NOISE #############################
    img_feature = clip_model.encode_image(torch.nn.functional.interpolate(\
                                           output_previous,(224,224)))
    img_feature_delta = img_feature - img_feature_old
    
    loss = nn.CosineSimilarity()(img_feature_delta,text_feature_delta)
    # loss = torch.var(output_gt) - torch.var(output_previous)
    
    loss.backward(retain_graph=True)
    print('loss: ',loss)

    ########################### ATTACK NOISE #############################


    optimal_perturbation_style = eps * torch.sign(batch['style'].grad) # inf norm, attack noise
    optimal_perturbation_content = eps * torch.sign(batch['content'].grad) # inf norm, attack noise
    eps *= # decay rate

    batch['style'].grad.zero_()
    batch['content'].grad.zero_()

    optimal_perturbation_style_old = batch['noise_style']
    optimal_perturbation_content_old = batch['noise_content']

    optimal_perturbation_style_new = torch.clip(optimal_perturbation_style_old + optimal_perturbation_style,\
                                                -0.01, 0.01) # manually set
    optimal_perturbation_content_new = torch.clip(optimal_perturbation_content_old + optimal_perturbation_content,\
                                                -0.15, 0.15) # manually set




    batch['style'] = batch['style'] - optimal_perturbation_style_old # clean input
    batch['content'] = batch['content'] - optimal_perturbation_content_old

    batch['noise_style'] = optimal_perturbation_style_new
    batch['noise_content'] = optimal_perturbation_content_new

    # save output

    output_adv= network(batch['content'].clone().detach() + batch['noise_content'],\
                        batch['style'].clone().detach() + batch['noise_style'] )


    output_name_adv = '{:s}/{:s}_stylized_{:s}_{:s}_{:s}{:s}'.format(
        output_path, splitext(basename(content_path))[0],
        splitext(basename(style_path))[0], word if word is not None else '', str(i), save_ext
    )
    save_image(output_adv, output_name_adv)